🔧 **Setup Required**: Before running this notebook, please follow the [setup instructions](../README.md#setup-instructions) to configure your environment and API keys.

# Knowledge Graph and Synthetic Data Generation Pipelines

This notebook demonstrates how to build comprehensive pipelines for:
1. **Knowledge Graph Creation** - Converting documents into structured knowledge representations
2. **Synthetic Test Data Generation** - Creating question-answer pairs for evaluation
3. **Multi-Source Processing** - Working with PDFs, web content, and other document types

## Learning Objectives

By the end of this notebook, you will understand:
- How to build end-to-end Haystack pipelines for knowledge extraction
- The relationship between knowledge graphs and test data generation
- Best practices for processing different document formats
- How to evaluate and validate synthetic datasets


## Part 1: PDF Processing Pipeline

### Overview
In this section, we'll build a comprehensive pipeline that:
1. **Extracts content** from PDF files using Haystack's PyPDFToDocument converter
2. **Preprocesses the text** with cleaning and splitting components
3. **Creates a knowledge graph** from the processed documents
4. **Generates synthetic test data** using the knowledge graph

### Key Components
- **PyPDFToDocument**: Converts PDF files to Haystack Document objects
- **DocumentCleaner**: Removes extra whitespaces and empty lines
- **DocumentSplitter**: Breaks documents into manageable chunks
- **KnowledgeGraphGenerator**: Creates structured knowledge representations
- **SyntheticTestGenerator**: Produces question-answer pairs for evaluation

### Why This Approach?
Using knowledge graphs as an intermediate step improves the quality of synthetic test generation because:
- Knowledge graphs capture relationships between entities
- They provide structured context for question generation
- The resulting questions are more coherent and factually grounded

In [1]:
import os
from dotenv import load_dotenv
from haystack import Pipeline
from haystack.components.converters import PyPDFToDocument
from haystack.components.preprocessors import (
    DocumentCleaner,
    DocumentSplitter)
from pathlib import Path
from scripts.knowledge_graph_component import KnowledgeGraphGenerator,\
                                                DocumentToLangChainConverter
from scripts.synthetic_test_components import SyntheticTestGenerator,\
                                                TestDatasetSaver
                                                    
# Load environment variables
load_dotenv("./.env")

        
# Create pipeline components
pdf_converter = PyPDFToDocument()
doc_cleaner = DocumentCleaner(remove_empty_lines=True,
                                remove_extra_whitespaces=True)
doc_splitter = DocumentSplitter(split_by="sentence",
                                split_length=50,
                                split_overlap=5)
doc_converter = DocumentToLangChainConverter()
kg_generator = KnowledgeGraphGenerator(apply_transforms=True)


test_generator = SyntheticTestGenerator(
    testset_size=10,  
    llm_model="gpt-4o-mini",
    query_distribution=[
        ("single_hop", 0.25), 
        ("multi_hop_specific", 0.25),
        ("multi_hop_abstract", 0.5)
    ],
    # Optional: Add max_testset_size=5 if you want to limit due to API constraints
    # max_testset_size=5  # Uncomment this line if you experience API timeouts
)
test_saver = TestDatasetSaver("data_for_eval/synthetic_tests_10_from_pdf.csv")

# Create pipeline
pipeline = Pipeline()
pipeline.add_component("pdf_converter", pdf_converter)
pipeline.add_component("doc_cleaner", doc_cleaner)
pipeline.add_component("doc_splitter", doc_splitter)
pipeline.add_component("doc_converter", doc_converter)
pipeline.add_component("kg_generator", kg_generator)
pipeline.add_component("test_generator", test_generator)
pipeline.add_component("test_saver", test_saver)

# Connect components in sequence
pipeline.connect("pdf_converter.documents", "doc_cleaner.documents")
pipeline.connect("doc_cleaner.documents", "doc_splitter.documents")
pipeline.connect("doc_splitter.documents", "doc_converter.documents")
pipeline.connect("doc_converter.langchain_documents", "kg_generator.documents")
pipeline.connect("kg_generator.knowledge_graph", "test_generator.knowledge_graph")
pipeline.connect("doc_converter.langchain_documents", "test_generator.documents")
pipeline.connect("test_generator.testset", "test_saver.testset")


/Users/laurafunderburk/Documents/GitHub/Building-Natural-Language-Pipelines/ch5/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🚅 Components
  - pdf_converter: PyPDFToDocument
  - doc_cleaner: DocumentCleaner
  - doc_splitter: DocumentSplitter
  - doc_converter: DocumentToLangChainConverter
  - kg_generator: KnowledgeGraphGenerator
  - test_generator: SyntheticTestGenerator
  - test_saver: TestDatasetSaver
🛤️ Connections
  - pdf_converter.documents -> doc_cleaner.documents (list[Document])
  - doc_cleaner.documents -> doc_splitter.documents (list[Document])
  - doc_splitter.documents -> doc_converter.documents (list[Document])
  - doc_converter.langchain_documents -> kg_generator.documents (List[Document])
  - doc_converter.langchain_documents -> test_generator.documents (List[Document])
  - kg_generator.knowledge_graph -> test_generator.knowledge_graph (KnowledgeGraph)
  - test_generator.testset -> test_saver.testset (DataFrame)

In [2]:
# Prepare input data - convert PDF files to ByteStream objects
pdf_sources = [Path("./data_for_indexing/howpeopleuseai.pdf")]
result = pipeline.run({
            "pdf_converter": {"sources": pdf_sources}
        })
        
print("\n📊 Pipeline Results:")
print(f"  📄 Documents Processed: {result['doc_converter']['document_count']}")
print(f"  🧠 Knowledge Graph Nodes: {result['kg_generator']['node_count']}")
print(f"  🧪 Test Cases Generated: {result['test_generator']['testset_size']}")
print(f"  🔧 Generation Method: {result['test_generator']['generation_method']}")

Generating Samples: 100%|██████████| 11/11 [00:06<00:00,  1.58it/s]



📊 Pipeline Results:
  📄 Documents Processed: 17
  🧠 Knowledge Graph Nodes: 17
  🧪 Test Cases Generated: 11
  🔧 Generation Method: knowledge_graph


### Understanding the Pipeline Architecture

The pipeline we're building follows this flow:

```
PDF File → PDF Converter → Document Cleaner → Document Splitter 
    ↓
Document Converter → Knowledge Graph Generator
    ↓                         ↓
Test Generator ← ← ← ← ← ← ← ←
    ↓
Test Dataset Saver
```

**Key Design Decisions:**

1. **Document Processing Chain**: We clean and split documents before knowledge graph generation to ensure high-quality input
2. **Dual Input to Test Generator**: Both the knowledge graph and original documents are provided to enable fallback generation methods
3. **Configurable Test Distribution**: We can control the types of questions generated (single-hop vs multi-hop)

**Pipeline Parameters Explained:**
- `testset_size=10`: Number of question-answer pairs to generate
- `split_length=50`: Number of sentences per document chunk
- `query_distribution`: Controls complexity of generated questions

In [3]:
pipeline.draw(path="./images/knowledgegraph_sdg_pipeline_pdf.png")

![](./images/knowledgegraph_sdg_pipeline_pdf.png)

In [4]:
import pandas as pd

# Load and display the generated synthetic tests
test_file_path = "data_for_eval/synthetic_tests_10_from_pdf.csv"

if os.path.exists(test_file_path):
    synthetic_tests_df = pd.read_csv(test_file_path)
    print("\n🧪 Synthetic Tests Sample:")
    print("First 5 rows:")
    display(synthetic_tests_df.head())
    print("Last 5 rows:")
    display(synthetic_tests_df.tail())
else:
    print("❌ Synthetic test file not found")


🧪 Synthetic Tests Sample:
First 5 rows:


,user_input,reference_contexts,reference,synthesizer_name
0,What role does Christopher Ong play in the res...,['NBER WORKING PAPER SERIES\nHOW PEOPLE USE CH...,Christopher Ong is one of the co-authors of th...,single_hop_specific_query_synthesizer
1,What role does David J. Deming play in the stu...,['ABSTRACT Despite the rapid adoption of LLM c...,David J. Deming is affiliated with Harvard Uni...,single_hop_specific_query_synthesizer
2,What are the different consumer plans availabl...,['to classify messages without any human seein...,OpenAI offers a variety of consumer plans for ...,single_hop_specific_query_synthesizer
3,What is the estimated economic surplus generat...,['<1-hop>\n\nto classify messages without any ...,The estimated economic surplus generated by US...,multi_hop_specific_query_synthesizer
4,What demographic trends in ChatGPT usage are h...,"['<1-hop>\n\nTeams, Enterprise, Education), wh...",Zao-Sanders (2025) highlights several demograp...,multi_hop_specific_query_synthesizer


Last 5 rows:


,user_input,reference_contexts,reference,synthesizer_name
6,What are the differences in ChatGPT usage for ...,['<1-hop>\n\n37% of messages are work-related\...,Users with different education levels exhibit ...,multi_hop_abstract_query_synthesizer
7,How do the conversation topics identified in u...,['<1-hop>\n\nFigure 9 disaggregates four of th...,The conversation topics identified in user int...,multi_hop_abstract_query_synthesizer
8,What percentage of work-related messages are s...,['<1-hop>\n\n37% of messages are work-related\...,Users in computer-related occupations send 57%...,multi_hop_abstract_query_synthesizer
9,What are the main conversation topics in ChatG...,['<1-hop>\n\nFigure 9 disaggregates four of th...,The main conversation topics in ChatGPT messag...,multi_hop_abstract_query_synthesizer
10,What percentage of work-related messages are c...,['<1-hop>\n\nNearly 35% of all work-related qu...,Nearly 56% of work-related queries are classif...,multi_hop_abstract_query_synthesizer


### Analyzing the Generated Test Dataset

Now let's examine the synthetic test data that was generated from our PDF processing pipeline.

**What to Look For:**
- **Question Quality**: Are the questions grammatically correct and meaningful?
- **Answer Accuracy**: Do the answers correctly reflect the source material?
- **Question Types**: Notice the variety of single-hop and multi-hop questions
- **Context Relevance**: Check if the reference contexts support the answers

**Common Question Types You'll See:**
1. **Single-hop questions**: Direct factual queries (e.g., "What is X?")
2. **Multi-hop specific**: Questions requiring connecting specific facts
3. **Multi-hop abstract**: Questions requiring broader reasoning across multiple concepts

## Part 2: Web Content Processing Pipeline

### Overview
In this section, we'll adapt our pipeline to work with web content instead of PDF files. This demonstrates the flexibility of Haystack pipelines and how the same knowledge graph generation approach can work across different content sources.

### Key Differences from PDF Processing
1. **LinkContentFetcher**: Retrieves content directly from URLs
2. **HTMLToDocument**: Converts HTML content to Haystack Documents
3. **Same Processing Chain**: The rest of the pipeline remains identical

### Real-World Applications
This approach is particularly useful for:
- **Documentation Analysis**: Processing online documentation and creating test datasets
- **Content Monitoring**: Regularly generating tests from updated web content  
- **Multi-Source Knowledge**: Combining web content with other document types
- **Research Applications**: Creating datasets from academic papers, blog posts, etc.

### Technical Considerations
- **Rate Limiting**: Be mindful of website rate limits when fetching content
- **Content Quality**: Web content may require more aggressive cleaning
- **Dynamic Content**: Some websites use JavaScript; static HTML fetching may miss content

In [5]:
from haystack.components.fetchers import LinkContentFetcher
from haystack.components.converters import HTMLToDocument

fetcher = LinkContentFetcher()
converter = HTMLToDocument()
doc_cleaner = DocumentCleaner(remove_empty_lines=True,
                            remove_extra_whitespaces=True)
doc_splitter = DocumentSplitter(split_by="sentence",
                                split_length=50,
                                split_overlap=5)
doc_converter = DocumentToLangChainConverter()
kg_generator = KnowledgeGraphGenerator(apply_transforms=True)
test_generator = SyntheticTestGenerator(
            testset_size=10,  
            llm_model="gpt-4o-mini",
            query_distribution=[
                ("single_hop", 0.25), 
                ("multi_hop_specific", 0.25),
                ("multi_hop_abstract", 0.5)
            ]
        )
test_saver = TestDatasetSaver("data_for_eval/synthetic_tests_10_from_html_page.csv")

# Create pipeline
pipeline = Pipeline()
pipeline.add_component("fetcher", fetcher)
pipeline.add_component("converter", converter)
pipeline.add_component("doc_cleaner", doc_cleaner)
pipeline.add_component("doc_splitter", doc_splitter)
pipeline.add_component("doc_converter", doc_converter)
pipeline.add_component("kg_generator", kg_generator)
pipeline.add_component("test_generator", test_generator)
pipeline.add_component("test_saver", test_saver)

# Connect components in sequence
pipeline.connect("fetcher.streams", "converter.sources")
pipeline.connect("converter.documents", "doc_cleaner.documents")
pipeline.connect("doc_cleaner.documents", "doc_splitter.documents")
pipeline.connect("doc_splitter.documents", "doc_converter.documents")
pipeline.connect("doc_converter.langchain_documents", "kg_generator.documents")
pipeline.connect("kg_generator.knowledge_graph", "test_generator.knowledge_graph")
pipeline.connect("doc_converter.langchain_documents", "test_generator.documents")
pipeline.connect("test_generator.testset", "test_saver.testset")

🚅 Components
  - fetcher: LinkContentFetcher
  - converter: HTMLToDocument
  - doc_cleaner: DocumentCleaner
  - doc_splitter: DocumentSplitter
  - doc_converter: DocumentToLangChainConverter
  - kg_generator: KnowledgeGraphGenerator
  - test_generator: SyntheticTestGenerator
  - test_saver: TestDatasetSaver
🛤️ Connections
  - fetcher.streams -> converter.sources (list[ByteStream])
  - converter.documents -> doc_cleaner.documents (list[Document])
  - doc_cleaner.documents -> doc_splitter.documents (list[Document])
  - doc_splitter.documents -> doc_converter.documents (list[Document])
  - doc_converter.langchain_documents -> kg_generator.documents (List[Document])
  - doc_converter.langchain_documents -> test_generator.documents (List[Document])
  - kg_generator.knowledge_graph -> test_generator.knowledge_graph (KnowledgeGraph)
  - test_generator.testset -> test_saver.testset (DataFrame)

In [6]:
web_url = "https://haystack.deepset.ai/blog/haystack-2-release"

result = pipeline.run({
    "fetcher": {"urls": [web_url]}
})

print("\n📊 Pipeline Results:")
print(f"  📄 Documents Processed: {result['doc_converter']['document_count']}")
print(f"  🧠 Knowledge Graph Nodes: {result['kg_generator']['node_count']}")
print(f"  🧪 Test Cases Generated: {result['test_generator']['testset_size']}")
print(f"  🔧 Generation Method: {result['test_generator']['generation_method']}")

Generating Samples: 100%|██████████| 11/11 [00:07<00:00,  1.57it/s]



📊 Pipeline Results:
  📄 Documents Processed: 2
  🧠 Knowledge Graph Nodes: 2
  🧪 Test Cases Generated: 11
  🔧 Generation Method: knowledge_graph


### Web Pipeline Architecture

The web processing pipeline follows a similar structure but with adapted input components:

```
Web URL → Link Fetcher → HTML Converter → Document Cleaner → Document Splitter
    ↓
Document Converter → Knowledge Graph Generator  
    ↓                         ↓
Test Generator ← ← ← ← ← ← ← ←
    ↓
Test Dataset Saver
```

**Why This Works:**
- The knowledge graph generation is **content-agnostic** - it works the same whether input comes from PDFs, web pages, or other sources
- Document preprocessing steps ensure consistent quality regardless of input format
- The same test generation logic produces comparable quality across all sources

**Pipeline Reusability:**
Notice how we can reuse the same components (`doc_cleaner`, `doc_splitter`, `kg_generator`, etc.) with different input sources. This demonstrates the modularity and flexibility of Haystack's component architecture.

In [7]:
pipeline.draw(path="./images/knowledgegraph_sdg_pipeline_html.png")

![](./images/knowledgegraph_sdg_pipeline_html.png)

In [8]:
# Load and display the generated synthetic tests
test_file_path = "data_for_eval/synthetic_tests_10_from_html_page.csv"

if os.path.exists(test_file_path):
    synthetic_tests_df = pd.read_csv(test_file_path)
    print("\n🧪 Synthetic Tests Sample:")
    print("First 5 rows:")
    display(synthetic_tests_df.head())
    print("Last 5 rows:")
    display(synthetic_tests_df.tail())
else:
    print("❌ Synthetic test file not found")


🧪 Synthetic Tests Sample:
First 5 rows:


,user_input,reference_contexts,reference,synthesizer_name
0,What support will be provided for Haystack 1.0...,['Haystack 2.0: The Composable Open-Source LLM...,"After the release of Haystack 2.0, users of Ha...",single_hop_specific_query_synthesizer
1,What limitations did Haystack 1.0 have regardi...,['Composable and customizable Pipelines\nModer...,One important limitation in Haystack 1.0 is th...,single_hop_specific_query_synthesizer
2,What is Haystack 2.0?,['A common interface for storing data - A clea...,Haystack 2.0 is an advanced version of the AI ...,single_hop_specific_query_synthesizer
3,What limitations of Haystack 1.0 were addresse...,['<1-hop>\n\nComposable and customizable Pipel...,Haystack 1.0 had a significant limitation in t...,multi_hop_specific_query_synthesizer
4,What is Haystack and how does it help with dat...,['<1-hop>\n\nA common interface for storing da...,Haystack is an AI framework that provides a co...,multi_hop_specific_query_synthesizer


Last 5 rows:


,user_input,reference_contexts,reference,synthesizer_name
6,How does Haystack 2.0 enhance customization an...,"['<1-hop>\n\nThese include Chroma, Weaviate, P...",Haystack 2.0 enhances customization and flexib...,multi_hop_abstract_query_synthesizer
7,What are the key features of Haystack 2.0 that...,"['<1-hop>\n\nThese include Chroma, Weaviate, P...",Haystack 2.0 is designed to be a flexible and ...,multi_hop_abstract_query_synthesizer
8,What are the key features of Haystack 2.0 that...,"['<1-hop>\n\nThese include Chroma, Weaviate, P...",Haystack 2.0 is designed to be a flexible and ...,multi_hop_abstract_query_synthesizer
9,What are the key features of Haystack 2.0 that...,"['<1-hop>\n\nThese include Chroma, Weaviate, P...",Haystack 2.0 is designed to be a flexible and ...,multi_hop_abstract_query_synthesizer
10,What are the key features of Haystack 2.0 that...,"['<1-hop>\n\nThese include Chroma, Weaviate, P...",Haystack 2.0 is designed to be a flexible and ...,multi_hop_abstract_query_synthesizer


### Comparing Results Across Sources

Let's examine how the synthetic test generation performs when using web content versus PDF content.

**Expected Differences:**
- **Content Structure**: Web content may have different formatting and structure
- **Question Complexity**: Depending on the source material's complexity
- **Context Quality**: Web content might include navigation elements or ads that need filtering

**Quality Assessment Checklist:**
- [ ] Questions are grammatically correct
- [ ] Answers are factually accurate based on the source
- [ ] Context excerpts support the provided answers
- [ ] Questions test different levels of comprehension
- [ ] No duplicate or overly similar questions

## Part 3: Unified Branching Pipeline - PDF or Web Content

### Overview
In this section, we'll create a more sophisticated pipeline that can intelligently route inputs based on their type. This pipeline demonstrates:

1. **Input Flexibility**: Accept either PDF files OR web URLs as input
2. **Intelligent Routing**: Automatically determine the processing path based on input type  
3. **Unified Processing**: Both input types follow the same knowledge graph and synthetic data generation workflow
4. **Production-Ready Design**: Handle multiple input sources in a single, cohesive pipeline

### Key Components for Branching
- **Conditional Logic**: Python conditionals to determine input type
- **Modular Design**: Reusable component instances across different input paths
- **Error Handling**: Graceful handling of different input scenarios
- **Consistent Output**: Same output format regardless of input source

### Real-World Benefits
This approach is valuable because:
- **Operational Efficiency**: One pipeline handles multiple input scenarios
- **Maintenance Simplicity**: Single codebase for different input types  
- **Flexible Deployment**: Can be easily extended to handle additional input formats
- **Cost Optimization**: Shared components reduce resource requirements

### Advanced Branching Pipeline with FileTypeRouter

Now let's create an even more sophisticated version using Haystack's `FileTypeRouter` component, which can automatically detect input types and route them appropriately. This approach is more scalable and follows the example pattern you provided.

**Key Advantages of FileTypeRouter Approach:**
- **Automatic Detection**: No manual input type checking required
- **Extensibility**: Easy to add new input types (CSV, Word docs, etc.)
- **Production Ready**: Built-in error handling and type validation  
- **Performance**: Efficient routing without conditional logic overhead

In [12]:
from haystack.components.routers import FileTypeRouter
from haystack.components.joiners import DocumentJoiner

website = ["https://www.tableau.com/data-insights/ai/examples"]



# Initialize pipeline
advanced_pipeline = Pipeline()

# Core routing and joining components  
file_router = FileTypeRouter(mime_types=["text/plain", "application/pdf", "text/html"])
doc_joiner = DocumentJoiner()  # Joins documents from different branches

# Input converters for each file type
pdf_converter = PyPDFToDocument()
html_converter = HTMLToDocument()  
link_fetcher = LinkContentFetcher()

# Shared processing components
doc_cleaner = DocumentCleaner(remove_empty_lines=True, remove_extra_whitespaces=True)
doc_splitter = DocumentSplitter(split_by="sentence", split_length=50, split_overlap=5)
doc_converter = DocumentToLangChainConverter()
kg_generator = KnowledgeGraphGenerator(apply_transforms=True)
test_generator = SyntheticTestGenerator(
    testset_size=15,  # Larger test set for multiple sources
    llm_model="gpt-4o-mini",
    query_distribution=[
        ("single_hop", 0.3),
        ("multi_hop_specific", 0.3), 
        ("multi_hop_abstract", 0.4)
    ]
)
test_saver = TestDatasetSaver("data_for_eval/synthetic_tests_advanced_branching.csv")

# Add all components to pipeline
advanced_pipeline.add_component("file_router", file_router)
advanced_pipeline.add_component("link_fetcher", link_fetcher)
advanced_pipeline.add_component("pdf_converter", pdf_converter) 
advanced_pipeline.add_component("html_converter", html_converter)
advanced_pipeline.add_component("doc_joiner", doc_joiner)
advanced_pipeline.add_component("doc_cleaner", doc_cleaner)
advanced_pipeline.add_component("doc_splitter", doc_splitter)
advanced_pipeline.add_component("doc_converter", doc_converter)
advanced_pipeline.add_component("kg_generator", kg_generator)
advanced_pipeline.add_component("test_generator", test_generator)
advanced_pipeline.add_component("test_saver", test_saver)

# Connect file routing branches
advanced_pipeline.connect("file_router.application/pdf", "pdf_converter.sources") 
advanced_pipeline.connect("link_fetcher.streams", "html_converter.sources")

# Connect converters to joiner
advanced_pipeline.connect("pdf_converter.documents", "doc_joiner.documents")
advanced_pipeline.connect("html_converter.documents", "doc_joiner.documents")

# Connect main processing path
advanced_pipeline.connect("doc_joiner.documents", "doc_cleaner.documents")
advanced_pipeline.connect("doc_cleaner.documents", "doc_splitter.documents")
advanced_pipeline.connect("doc_splitter.documents", "doc_converter.documents")
advanced_pipeline.connect("doc_converter.langchain_documents", "kg_generator.documents")
advanced_pipeline.connect("kg_generator.knowledge_graph", "test_generator.knowledge_graph")
advanced_pipeline.connect("doc_converter.langchain_documents", "test_generator.documents")
advanced_pipeline.connect("test_generator.testset", "test_saver.testset")


🚅 Components
  - file_router: FileTypeRouter
  - link_fetcher: LinkContentFetcher
  - pdf_converter: PyPDFToDocument
  - html_converter: HTMLToDocument
  - doc_joiner: DocumentJoiner
  - doc_cleaner: DocumentCleaner
  - doc_splitter: DocumentSplitter
  - doc_converter: DocumentToLangChainConverter
  - kg_generator: KnowledgeGraphGenerator
  - test_generator: SyntheticTestGenerator
  - test_saver: TestDatasetSaver
🛤️ Connections
  - file_router.application/pdf -> pdf_converter.sources (list[Union[str, Path, ByteStream]])
  - link_fetcher.streams -> html_converter.sources (list[ByteStream])
  - pdf_converter.documents -> doc_joiner.documents (list[Document])
  - html_converter.documents -> doc_joiner.documents (list[Document])
  - doc_joiner.documents -> doc_cleaner.documents (list[Document])
  - doc_cleaner.documents -> doc_splitter.documents (list[Document])
  - doc_splitter.documents -> doc_converter.documents (list[Document])
  - doc_converter.langchain_documents -> kg_generator.docu

In [13]:
# Define inputs
pdf_file = Path("./data_for_indexing/howpeopleuseai.pdf")
web_url = "https://www.tableau.com/data-insights/ai/examples"

try:
    # Run pipeline with both input types
    result = advanced_pipeline.run({
        "file_router": {"sources": [pdf_file]},  # PDF input through FileTypeRouter
        "link_fetcher": {"urls": [web_url]}      # Web input through LinkContentFetcher
    })
    
    print("\n📊 Advanced Pipeline Results:")
    print(f"  📄 Total Documents Processed: {result['doc_converter']['document_count']}")
    print(f"  🧠 Knowledge Graph Nodes: {result['kg_generator']['node_count']}")
    print(f"  🧪 Test Cases Generated: {result['test_generator']['testset_size']}")
    print(f"  🔧 Generation Method: {result['test_generator']['generation_method']}")
    print(f"  💾 Combined output saved to: data_for_eval/synthetic_tests_advanced_branching.csv")
    
    # Show the power of the joiner
    print(f"\n🔗 Document Joiner successfully combined:")
    print(f"  - PDF content from: {pdf_file.name}")
    print(f"  - Web content from: {web_url}")
    print(f"  Into a unified knowledge graph and test dataset!")
    
except Exception as e:
    print(f"❌ Error in advanced pipeline: {str(e)}")
    print("This might be due to network issues or file access problems.")

Applying OverlapScoreBuilder: 100%|██████████| 1/1 [00:00<00:00, 68.63it/s]

Generating Samples: 100%|██████████| 16/16 [00:09<00:00,  1.76it/s]




📊 Advanced Pipeline Results:
  📄 Total Documents Processed: 20
  🧠 Knowledge Graph Nodes: 20
  🧪 Test Cases Generated: 16
  🔧 Generation Method: knowledge_graph
  💾 Combined output saved to: data_for_eval/synthetic_tests_advanced_branching.csv

🔗 Document Joiner successfully combined:
  - PDF content from: howpeopleuseai.pdf
  - Web content from: https://www.tableau.com/data-insights/ai/examples
  Into a unified knowledge graph and test dataset!


In [14]:
# Visualize the advanced branching pipeline architecture
advanced_pipeline.draw(path="./images/advanced_branching_kg_pipeline.png")


![](./images/advanced_branching_kg_pipeline.png)

In [16]:
# Load and compare results from the advanced branching pipeline
advanced_test_file = "data_for_eval/synthetic_tests_advanced_branching.csv"

if os.path.exists(advanced_test_file):
    advanced_tests_df = pd.read_csv(advanced_test_file)
    
    display(advanced_tests_df.head())
    display(advanced_tests_df.tail())

,user_input,reference_contexts,reference,synthesizer_name
0,How does Yahoo utilize AI in its search engine...,['You may or may not be aware of how pervasive...,"Yahoo, as a popular search engine, utilizes AI...",single_hop_specific_query_synthesizer
1,How Facebook use AI for keep users engaged and...,['Social media\nSocial media platforms are ano...,"Facebook, like other major social media platfo...",single_hop_specific_query_synthesizer
2,How does Grammarly help with text editing and ...,['Some examples of industrial robots include:\...,Grammarly is an online text editor that takes ...,single_hop_specific_query_synthesizer
3,What are some applications of AI in healthcare?,['Fraud prevention\nIf you have an account wit...,"AI has many uses in the field of healthcare, i...",single_hop_specific_query_synthesizer
4,Who is Zoe Hitzig and what is her affiliation?,['NBER WORKING PAPER SERIES\nHOW PEOPLE USE CH...,Zoe Hitzig is affiliated with OpenAI and the H...,single_hop_specific_query_synthesizer


,user_input,reference_contexts,reference,synthesizer_name
11,How does the training of language models relat...,"['<1-hop>\n\nOuyang, Long, Jeff Wu, Xu Jiang, ...","The training of language models, as discussed ...",multi_hop_abstract_query_synthesizer
12,What is the relationship between ChatGPT usage...,['<1-hop>\n\n37% of messages are work-related\...,ChatGPT usage for work-related messages varies...,multi_hop_abstract_query_synthesizer
13,What are the classifier prompts used for train...,"['<1-hop>\n\nOuyang, Long, Jeff Wu, Xu Jiang, ...",The classifier prompts used for training langu...,multi_hop_abstract_query_synthesizer
14,How generative AI is used for task automation ...,"['<1-hop>\n\nOuyang, Long, Jeff Wu, Xu Jiang, ...",Generative AI is used for task automation by p...,multi_hop_abstract_query_synthesizer
15,How does the usage of ChatGPT for work-related...,['<1-hop>\n\n37% of messages are work-related\...,The usage of ChatGPT for work-related messages...,multi_hop_abstract_query_synthesizer


### Branching Pipeline Architecture Analysis

The advanced branching pipeline we just built demonstrates several key architectural patterns:

#### Pipeline Flow
```
Input Sources (PDF + Web URL)
    ↓                    ↓
FileTypeRouter    LinkContentFetcher
    ↓                    ↓  
PDFConverter      HTMLConverter
    ↓                    ↓
    └── DocumentJoiner ──┘
            ↓
    Document Processing Chain
    (Cleaner → Splitter → Converter)
            ↓
    Knowledge Graph Generator
            ↓
    Synthetic Test Generator  
            ↓
    Test Dataset Saver
```

#### Key Architectural Benefits

1. **Modularity**: Each component has a single responsibility and can be reused
2. **Flexibility**: Easy to add new input types (CSV, Word docs, etc.) 
3. **Scalability**: DocumentJoiner allows processing multiple sources simultaneously
4. **Consistency**: Same processing logic regardless of input source
5. **Error Isolation**: Problems with one input source don't affect others

#### Production Considerations

**Advantages of This Approach:**
- **Unified Output**: Single knowledge graph and test dataset from multiple sources
- **Rich Context**: Cross-referencing information between different document types
- **Operational Efficiency**: One pipeline deployment handles multiple scenarios
- **Quality Improvement**: More diverse training data leads to better synthetic questions

**When to Use Branching Pipelines:**
- Processing heterogeneous document collections
- Building comprehensive knowledge bases from multiple sources
- Creating robust test datasets that cover various content types
- Implementing production pipelines that need input flexibility

## Summary and Next Steps

### What We've Learned

In this notebook, we explored:

1. **Knowledge Graph-Driven Test Generation**: How structured knowledge representations improve synthetic data quality
2. **Multi-Source Processing**: Adapting the same pipeline architecture for different input types (PDFs, web content)
3. **Pipeline Modularity**: Reusing components across different use cases while maintaining consistency
4. **Quality Assessment**: Evaluating synthetic test datasets for accuracy and usefulness

### Key Takeaways

- **Knowledge graphs act as a quality filter** for test generation, producing more coherent and factually grounded questions
- **Haystack's component architecture** enables easy adaptation between different content sources
- **Preprocessing matters** - cleaning and splitting documents appropriately affects downstream quality
- **Synthetic test generation** can scale evaluation efforts but requires careful quality validation

### Production Considerations

When moving to production, consider:

1. **Quality Control**: Implement automated quality checks (see the quality control components in other notebooks)
2. **Scalability**: Use batch processing for large document collections
3. **Monitoring**: Track generation success rates and quality metrics over time
4. **Cost Management**: Balance test quantity with API usage costs
5. **Validation**: Always human-review a sample of generated tests before deployment
